In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython import display
import numpy as np
%matplotlib inline
import pygmo as pg
from pygmo import algorithm
import os
import sys
import json
from orbsim.r3b_2d import UNIT_TIME
from orbsim.r3b_2d.analyticals import *
from orbsim.r3b_2d.simulators import launch_sim
from orbsim.plotting import orbitplot2d, orbitplot_non_inertial
import time
from random import shuffle
from numba import jit
import math
from math import pi

pi8=pi/8
pi4=pi/4
pi2=pi/2

In [ ]:
psis=[]
paths=[]
title="long_leto_lyapunov"
N=5
for i in range(N):
    psi = [3.794182930145708, 0.023901745288554, 3.090702702702703+i/1e3]
    path = launch_sim(psi,max_iter=1e7, duration=200/UNIT_TIME)
    psis.append(psi)
    paths.append(path)


In [ ]:
for i in range(len(paths)):
    orbitplot2d(paths[i],psis[i],title=f"{title}_{i}")
    orbitplot_non_inertial(paths[i],psis[i],title=f"{title}_{i}")

In [ ]:
lyaps=[]
for a in range(N):
    for b in range(N):
        print(f"comparing {a} and {b}")
        if a==b or a>b:
            continue
        lyap=[]
        _a=np.array(paths[a][1]).T
        _b=np.array(paths[b][1]).T
        xas=_a[0]
        yas=_a[1]
        xbs=_b[0]
        ybs=_b[1]
        hs=(_a[4],_b[4])
        print(f"length of the trajectory coordinate arrays: {a}: {len(xas)}, {b}: {len(xbs)}")
        
        ts=[None,None]
        for l in range(len(hs)):
            idxs=[]
            tally=0
            for i in range(len(hs[l])): # each time step h, check whether the little tally has reached our threshold.
                h=hs[l][i]              # if it has, take that index as a time step
                tally+=h
                if tally>=1.5e-5:
                    idxs.append(i)
                    tally=0
            ts[l] = idxs
            
        _,min_ts = min([(x[-1], x) for x in ts])
        print(f"time steps standardized: comparing at {len(min_ts)} points on the trajectory. Last point will be at {max(min_ts)}")
        for i in range(len(min_ts)):
            idx=min_ts[i]
            lyap.append(sqrt((xas[idx]-xbs[idx])**2+(yas[idx]-ybs[idx])**2))
        lyaps.append(lyap)
print(len(lyaps))

In [ ]:
plt.plot(range(len(lyaps[5])),lyaps[5])

In [ ]:
for i in range(len(lyaps)):
    lyap=lyaps[i][1:]
    loglyap=[np.log(x) for x in lyap]
    plt.plot(range(len(loglyap)),loglyap)

In [ ]:
lyap=lyaps[0][1:]
loglyap=[np.log(x) for x in lyap]
plt.plot(range(len(loglyap)),loglyap)
plt.plot(range(len(line)),line)

In [ ]:
ts=[[1,1],[2]]
_,min_ts = max([(len(x), x) for x in ts])
min_ts

In [ ]:
line

In [ ]:
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(range(len(loglyap)),loglyap)

In [ ]:
line=[slope*range(len(loglyap))+intercept]